In [1]:
import math
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pmdarima as pm

/var/folders/bt/nlh_3ptx6znf2gx6qszlwdxm0000gn/T/ipykernel_1008/2074063697.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df = pd.read_csv("../Data/RawTrafficData.csv", index_col=0)
df.index = pd.to_datetime(df.index)
train = df.value[:800] # All models are trained on the first 800 instances (online learning is only evaluated after instance nr 800 to have fair comparison)
test = df.value[801:]

In [ ]:
%%timeit -r 1 -n 100

p, d, q = 6, 0, 6
P, D, Q, s = 2, 1, 2, 24 

sarima_model = sm.tsa.SARIMAX(train, order=(p, d, q), seasonal_order=(P, D, Q, s), initialization='approximate_diffuse')
results = sarima_model.fit()
forecast_steps = len(test)  
forecast = results.get_forecast(steps=forecast_steps)
forecast_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int(alpha=0.05)

ground_truth_values = test
anomaly_scores = np.zeros(len(ground_truth_values))

for i, true_value in enumerate(ground_truth_values):
    lower_bound = confidence_intervals.iloc[i, 0] 
    upper_bound = confidence_intervals.iloc[i, 1]
    prediction = (lower_bound + upper_bound) / 2
    
    threshold = np.abs(prediction-upper_bound) * 6
    error = np.abs(true_value - prediction)

    if error >= threshold:
        anomaly_scores[i] = 1.0
    else:
        anomaly_scores[i] = error / threshold